In [1]:
import numpy as np

In [2]:
# initialize tensors
d = 2
chi = 10

A = np.random.rand(chi, d, chi)
B = np.random.rand(chi, d, chi)
sAB = np.ones(chi) / np.sqrt(chi)  # set trivial initial weights
sBA = np.ones(chi) / np.sqrt(chi)  # set trivial initial weights

In [3]:
from numpy import linalg as LA
from ncon import ncon

""" Contract infinite MPS from the left for environment tensor sigBA """

sigBA = np.random.rand(chi, chi)  # initialize random starting point
tol = 1e-10  # set convergence tolerance

# define the tensor network
tensors = [np.diag(sBA), np.diag(sBA), A, A.conj(), np.diag(sAB),
           np.diag(sAB), B, B.conj()]
labels = [[1,2],[1,3],[2,4],[3,5,6],[4,5,7],[6,8],[7,9],[8,10,-1],[9,10,-2]]

for k in range(1000):
    sigBA_new = ncon([sigBA, *tensors], labels)  # contract transfer operator
    sigBA_new = sigBA_new / np.trace(sigBA_new)  # normalize
    if LA.norm(sigBA - sigBA_new) < tol:  # check convergence
        print('success!')
        break
    else:
        print('iter: %d, diff: %e' % (k, LA.norm(sigBA - sigBA_new)))
        sigBA = sigBA_new

iter: 0, diff: 5.001624e+00
iter: 1, diff: 7.274854e-03
iter: 2, diff: 1.610530e-04
iter: 3, diff: 2.631970e-06
iter: 4, diff: 4.679026e-08
iter: 5, diff: 8.840484e-10
success!


In [4]:
from scipy.sparse.linalg import LinearOperator

In [5]:
from scipy.sparse.linalg import eigs

In [6]:
def left_contract_MPS(sigBA, sBA, A, sAB, B):
    """ Contract an infinite 2-site unit cell from the left for the environment
    density matrices sigBA (B-A link) and sigAB (A-B link)"""

    # initialize the starting vector
    chiBA = A.shape[0]
    if sigBA.shape[0] == chiBA:
        v0 = sigBA.reshape(np.prod(sigBA.shape))
    else:
        v0 = (np.eye(chiBA) / chiBA).reshape(chiBA**2)

    # define network for transfer operator contract
    tensors = [np.diag(sBA), np.diag(sBA), A, A.conj(), np.diag(sAB),
             np.diag(sAB), B, B.conj()]
    labels = [[1, 2], [1, 3], [2, 4], [3, 5, 6], [4, 5, 7], [6, 8], [7, 9],
            [8, 10, -1], [9, 10, -2]]

    # define function for boundary contraction and pass to eigs
    def left_iter(sigBA):
        return ncon([sigBA.reshape([chiBA, chiBA]), *tensors], labels).reshape([chiBA**2, 1])

    # returns k=1 eigenvalues and eigenvectors
    # matvec implements A * v
    Dtemp, sigBA = eigs(LinearOperator((chiBA**2, chiBA**2), matvec=left_iter),
                      k=1, which='LM', v0=v0, tol=1e-10)

    # normalize the environment density matrix sigBA
    if np.isrealobj(A):
        sigBA = np.real(sigBA)
    sigBA = sigBA.reshape(chiBA, chiBA)
    sigBA = 0.5 * (sigBA + np.conj(sigBA.T))
    sigBA = sigBA / np.trace(sigBA)

    # compute density matric sigAB for A-B link
    sigAB = ncon([sigBA, np.diag(sBA), np.diag(sBA), A, np.conj(A)],
               [[1, 2], [1, 3], [2, 4], [3, 5, -1], [4, 5, -2]])
    sigAB = sigAB / np.trace(sigAB)

    return sigBA, sigAB

In [7]:
sigBA = np.random.rand(chi, chi)

In [8]:
sigBA

array([[0.62404588, 0.22924408, 0.5579828 , 0.09736434, 0.96444732,
        0.7521296 , 0.67136081, 0.09433449, 0.25249477, 0.59573955],
       [0.61871567, 0.08837569, 0.61149002, 0.75732943, 0.21719081,
        0.83380599, 0.24036949, 0.24730341, 0.24989204, 0.51349704],
       [0.60422918, 0.77182437, 0.33257777, 0.02919485, 0.12914248,
        0.89773795, 0.34796173, 0.43712365, 0.28038085, 0.2107988 ],
       [0.5447179 , 0.3465992 , 0.05423294, 0.02126208, 0.44435301,
        0.89602969, 0.73167539, 0.61322337, 0.81574462, 0.05428966],
       [0.32618218, 0.89389167, 0.16367863, 0.24107989, 0.05790163,
        0.38939167, 0.36027595, 0.02681891, 0.01179469, 0.56835106],
       [0.42689975, 0.35801887, 0.65839229, 0.2031071 , 0.06755075,
        0.11473066, 0.72441427, 0.31970776, 0.90331484, 0.80484732],
       [0.77120577, 0.46446693, 0.70042678, 0.83166075, 0.00410862,
        0.58014442, 0.59907281, 0.06953435, 0.285995  , 0.78970512],
       [0.82527256, 0.88842747, 0.0664644

In [9]:
sigBA, sigAB = left_contract_MPS(sigBA, sBA, A, sAB, B)

In [10]:
sigBA

array([[0.10192615, 0.09996959, 0.10676374, 0.10354848, 0.08674204,
        0.09497854, 0.11382934, 0.07734062, 0.08257493, 0.08951825],
       [0.09996959, 0.10448723, 0.1082938 , 0.11690309, 0.09569765,
        0.10065855, 0.11579541, 0.08555455, 0.0808847 , 0.08778496],
       [0.10676374, 0.1082938 , 0.11386193, 0.11704398, 0.09678998,
        0.10370114, 0.12149333, 0.08642589, 0.08645858, 0.09376437],
       [0.10354848, 0.11690309, 0.11704398, 0.14211832, 0.11356385,
        0.11461175, 0.1254465 , 0.10181558, 0.08357122, 0.09090135],
       [0.08674204, 0.09569765, 0.09678998, 0.11356385, 0.09139747,
        0.09329247, 0.10368613, 0.08186896, 0.07008393, 0.07615194],
       [0.09497854, 0.10065855, 0.10370114, 0.11461175, 0.09329247,
        0.09742025, 0.11083838, 0.08346256, 0.07678494, 0.08340073],
       [0.11382934, 0.11579541, 0.12149333, 0.1254465 , 0.10368613,
        0.11083838, 0.12986159, 0.09259202, 0.0921323 , 0.09995437],
       [0.07734062, 0.08555455, 0.0864258

In [11]:
sigAB

array([[0.11576153, 0.1203462 , 0.10191229, 0.08381631, 0.10790192,
        0.1217157 , 0.08890222, 0.09175964, 0.11554757, 0.1071449 ],
       [0.1203462 , 0.13164456, 0.10894084, 0.08971005, 0.12001928,
        0.12824036, 0.09223943, 0.09828198, 0.12316183, 0.1148095 ],
       [0.10191229, 0.10894084, 0.09137765, 0.07530581, 0.09862386,
        0.10813235, 0.07810156, 0.08209051, 0.10317648, 0.09593392],
       [0.08381631, 0.08971005, 0.07530581, 0.06210923, 0.08129917,
        0.08904404, 0.06419227, 0.06758199, 0.08493741, 0.07902555],
       [0.10790192, 0.12001928, 0.09862386, 0.08129917, 0.11014874,
        0.11556947, 0.08259892, 0.08908091, 0.11138544, 0.10417756],
       [0.1217157 , 0.12824036, 0.10813235, 0.08904404, 0.11556947,
        0.12857212, 0.09336101, 0.09725088, 0.12233424, 0.11363948],
       [0.08890222, 0.09223943, 0.07810156, 0.06419227, 0.08259892,
        0.09336101, 0.06831421, 0.070375  , 0.08863114, 0.08213194],
       [0.09175964, 0.09828198, 0.0820905

In [12]:
def orthog_MPS(sigBA, muBA, B, sBA, A, dtol=1e-12):
    """ set the MPS gauge across B-A link to the canonical form """

    # diagonalize left environment
    dtemp, utemp = LA.eigh(sigBA)
    chitemp = sum(dtemp > dtol)
    DL = dtemp[range(-1, -chitemp - 1, -1)]
    UL = utemp[:, range(-1, -chitemp - 1, -1)]

    # diagonalize right environment
    dtemp, utemp = LA.eigh(muBA)
    chitemp = sum(dtemp > dtol)
    DR = dtemp[range(-1, -chitemp - 1, -1)]
    UR = utemp[:, range(-1, -chitemp - 1, -1)]

    # compute new weights for B-A link
    weighted_mat = (np.diag(np.sqrt(DL)) @ UL.T @ np.diag(sBA)
                  @ UR @ np.diag(np.sqrt(DR)))
    UBA, stemp, VhBA = LA.svd(weighted_mat, full_matrices=False)
    sBA = stemp / LA.norm(stemp)

    # build x,y gauge change matrices, implement gauge change on A and B
    x = np.conj(UL) @ np.diag(1 / np.sqrt(DL)) @ UBA
    y = np.conj(UR) @ np.diag(1 / np.sqrt(DR)) @ VhBA.T
    A = ncon([y, A], [[1, -1], [1, -2, -3]])
    B = ncon([B, x], [[-1, -2, 2], [2, -3]])

    return B, sBA, A

Define the Hamiltonian.

In [13]:
from scipy.linalg import expm

# define a Hamiltonian (here the quantum XX model)
sX = np.array([[0, 1], [1, 0]])
sY = np.array([[0, -1j], [1j, 0]])
hamAB = (np.real(np.kron(sX, sX) + np.kron(sY, sY))).reshape(2, 2, 2, 2)
hamBA = hamAB

# exponentiate Hamiltonian
tau = 0.1  # set time-step
evotype = 'imag'  # set imaginary time evolution
if evotype == "real":
    gateAB = expm(1j * tau * hamAB.reshape(d**2, d**2)).reshape(d, d, d, d)
    gateBA = expm(1j * tau * hamBA.reshape(d**2, d**2)).reshape(d, d, d, d)
elif evotype == "imag":
    gateAB = expm(-tau * hamAB.reshape(d**2, d**2)).reshape(d, d, d, d)
    gateBA = expm(-tau * hamBA.reshape(d**2, d**2)).reshape(d, d, d, d)

In [14]:
gateAB

array([[[[ 1.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  1.02006676],
         [-0.201336  ,  0.        ]]],


       [[[ 0.        , -0.201336  ],
         [ 1.02006676,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  1.        ]]]])

In [15]:
def apply_gate_MPS(gateAB, A, sAB, B, sBA, chi, stol=1e-7):
    """ apply a gate to an MPS across and a A-B link. Truncate the MPS back to
    some desired dimension chi"""

    # ensure singular values are above tolerance threshold
    sBA_trim = sBA * (sBA > stol) + stol * (sBA < stol)

    # contract gate into the MPS, then deompose composite tensor with SVD
    d = A.shape[1]
    chiBA = sBA_trim.shape[0]
    tensors = [np.diag(sBA_trim), A, np.diag(sAB), B, np.diag(sBA_trim), gateAB]
    connects = [[-1, 1], [1, 5, 2], [2, 4], [4, 6, 3], [3, -4], [-2, -3, 5, 6]]
    nshape = [d * chiBA, d * chiBA]
    utemp, stemp, vhtemp = LA.svd(ncon(tensors, connects).reshape(nshape),
                                full_matrices=False)

    # truncate to reduced dimension
    chitemp = min(chi, len(stemp))
    utemp = utemp[:, range(chitemp)].reshape(sBA_trim.shape[0], d * chitemp)
    vhtemp = vhtemp[range(chitemp), :].reshape(chitemp * d, chiBA)

    # remove environment weights to form new MPS tensors A and B
    A = (np.diag(1 / sBA_trim) @ utemp).reshape(sBA_trim.shape[0], d, chitemp)
    B = (vhtemp @ np.diag(1 / sBA_trim)).reshape(chitemp, d, chiBA)

    # new weights
    sAB = stemp[range(chitemp)] / LA.norm(stemp[range(chitemp)])

    return A, sAB, B

Putting it all together:

In [16]:
import numpy as np
from numpy import linalg as LA
from scipy.linalg import expm
from scipy.sparse.linalg import LinearOperator, eigs
from ncon import ncon
from typing import Optional


def doTEBD(hamAB: np.ndarray,
           hamBA: np.ndarray,
           A: np.ndarray,
           B: np.ndarray,
           sAB: np.ndarray,
           sBA: np.ndarray,
           chi: int,
           tau: float,
           evotype: Optional[str] = 'imag',
           numiter: Optional[int] = 1000,
           midsteps: Optional[int] = 10,
           E0: Optional[float] = 0.0):
    """
    Implementation of time evolution (real or imaginary) for MPS with 2-site unit
    cell (A-B), based on TEBD algorithm.
    Args:
    hamAB: nearest neighbor Hamiltonian coupling for A-B sites.
    hamBA: nearest neighbor Hamiltonian coupling for B-A sites.
    A: MPS tensor for A-sites of lattice.
    B: MPS tensor for B-sites of lattice.
    sAB: vector of weights for A-B links.
    sBA: vector of weights for B-A links.
    chi: maximum bond dimension of MPS.
    tau: time-step of evolution.
    evotype: set real (evotype='real') or imaginary (evotype='imag') evolution.
    numiter: number of time-step iterations to take.
    midsteps: number of time-steps between re-orthogonalization of the MPS.
    E0: specify the ground energy (if known).
    Returns:
    np.ndarray: MPS tensor for A-sites;
    np.ndarray: MPS tensor for B-sites;
    np.ndarray: vector sAB of weights for A-B links.
    np.ndarray: vector sBA of weights for B-A links.
    np.ndarray: two-site reduced density matrix rhoAB for A-B sites
    np.ndarray: two-site reduced density matrix rhoAB for B-A sites
    """
    # exponentiate Hamiltonian
    d = A.shape[1]
    if evotype == "real":
        gateAB = expm(1j * tau * hamAB.reshape(d**2, d**2)).reshape(d, d, d, d)
        gateBA = expm(1j * tau * hamBA.reshape(d**2, d**2)).reshape(d, d, d, d)
    elif evotype == "imag":
        gateAB = expm(-tau * hamAB.reshape(d**2, d**2)).reshape(d, d, d, d)
        gateBA = expm(-tau * hamBA.reshape(d**2, d**2)).reshape(d, d, d, d)

    # initialize environment matrices
    sigBA = np.eye(A.shape[0]) / A.shape[0]
    muAB = np.eye(A.shape[2]) / A.shape[2]

    for k in range(numiter + 1):
        if np.mod(k, midsteps) == 0 or (k == numiter):
#             """ Bring MPS to normal form """

#             # contract MPS from left and right
#             sigBA, sigAB = left_contract_MPS(sigBA, sBA, A, sAB, B)
#             muAB, muBA = right_contract_MPS(muAB, sBA, A, sAB, B)

#             # orthogonalise A-B and B-A links
#             B, sBA, A = orthog_MPS(sigBA, muBA, B, sBA, A)
#             A, sAB, B = orthog_MPS(sigAB, muAB, A, sAB, B)

#             # normalize the MPS tensors
#             A_norm = np.sqrt(ncon([np.diag(sBA**2), A, np.conj(A), np.diag(sAB**2)],
#                             [[1, 3], [1, 4, 2], [3, 4, 5], [2, 5]]))
#             A = A / A_norm
#             B_norm = np.sqrt(ncon([np.diag(sAB**2), B, np.conj(B), np.diag(sBA**2)],
#                             [[1, 3], [1, 4, 2], [3, 4, 5], [2, 5]]))
#             B = B / B_norm

            """ Compute energy and display """

            # compute 2-site local reduced density matrices
            rhoAB, rhoBA = loc_density_MPS(A, sAB, B, sBA)

            # evaluate the energy
            energyAB = ncon([hamAB, rhoAB], [[1, 2, 3, 4], [1, 2, 3, 4]])
            energyBA = ncon([hamBA, rhoBA], [[1, 2, 3, 4], [1, 2, 3, 4]])
            energy = 0.5 * (energyAB + energyBA)

            chitemp = min(A.shape[0], B.shape[0])
            enDiff = energy - E0
            print('iteration: %d of %d, chi: %d, t-step: %f, energy: %f, '
                'energy error: %e' % (k, numiter, chitemp, tau, energy, enDiff))

        """ Do evolution of MPS through one time-step """
        if k < numiter:
            # apply gate to A-B link
            A, sAB, B = apply_gate_MPS(gateAB, A, sAB, B, sBA, chi)

            # apply gate to B-A link
            B, sBA, A = apply_gate_MPS(gateBA, B, sBA, A, sAB, chi)

    rhoAB, rhoBA = loc_density_MPS(A, sAB, B, sBA)
    
    return A, B, sAB, sBA, rhoAB, rhoBA


def left_contract_MPS(sigBA, sBA, A, sAB, B):
    """ Contract an infinite 2-site unit cell from the left for the environment
    density matrices sigBA (B-A link) and sigAB (A-B link)"""

    # initialize the starting vector
    chiBA = A.shape[0]
    if sigBA.shape[0] == chiBA:
        v0 = sigBA.reshape(np.prod(sigBA.shape))
    else:
        v0 = (np.eye(chiBA) / chiBA).reshape(chiBA**2)

    # define network for transfer operator contract
    tensors = [np.diag(sBA), np.diag(sBA), A, A.conj(), np.diag(sAB),
             np.diag(sAB), B, B.conj()]
    labels = [[1, 2], [1, 3], [2, 4], [3, 5, 6], [4, 5, 7], [6, 8], [7, 9],
            [8, 10, -1], [9, 10, -2]]

    # define function for boundary contraction and pass to eigs
    def left_iter(sigBA):
        return ncon([sigBA.reshape([chiBA, chiBA]), *tensors],
                    labels).reshape([chiBA**2, 1])
    Dtemp, sigBA = eigs(LinearOperator((chiBA**2, chiBA**2), matvec=left_iter),
                      k=1, which='LM', v0=v0, tol=1e-10)

    # normalize the environment density matrix sigBA
    if np.isrealobj(A):
        sigBA = np.real(sigBA)
    sigBA = sigBA.reshape(chiBA, chiBA)
    sigBA = 0.5 * (sigBA + np.conj(sigBA.T))
    sigBA = sigBA / np.trace(sigBA)

    # compute density matric sigAB for A-B link
    sigAB = ncon([sigBA, np.diag(sBA), np.diag(sBA), A, np.conj(A)],
               [[1, 2], [1, 3], [2, 4], [3, 5, -1], [4, 5, -2]])
    sigAB = sigAB / np.trace(sigAB)

    return sigBA, sigAB


def right_contract_MPS(muAB, sBA, A, sAB, B):
    """ Contract an infinite 2-site unit cell from the right for the environment
    density matrices muAB (A-B link) and muBA (B-A link)"""

    # initialize the starting vector
    chiAB = A.shape[2]
    if muAB.shape[0] == chiAB:
        v0 = muAB.reshape(np.prod(muAB.shape))
    else:
        v0 = (np.eye(chiAB) / chiAB).reshape(chiAB**2)

    # define network for transfer operator contract
    tensors = [np.diag(sAB), np.diag(sAB), A, A.conj(), np.diag(sBA),
             np.diag(sBA), B, B.conj()]
    labels = [[1, 2], [3, 1], [5, 2], [6, 4, 3], [7, 4, 5], [8, 6], [10, 7],
            [-1, 9, 8], [-2, 9, 10]]

    # define function for boundary contraction and pass to eigs
    def right_iter(muAB):
        return ncon([muAB.reshape([chiAB, chiAB]), *tensors],
                    labels).reshape([chiAB**2, 1])
    Dtemp, muAB = eigs(LinearOperator((chiAB**2, chiAB**2), matvec=right_iter),
                     k=1, which='LM', v0=v0, tol=1e-10)

    # normalize the environment density matrix muAB
    if np.isrealobj(A):
        muAB = np.real(muAB)
    muAB = muAB.reshape(chiAB, chiAB)
    muAB = 0.5 * (muAB + np.conj(muAB.T))
    muAB = muAB / np.trace(muAB)

    # compute density matrix muBA for B-A link
    muBA = ncon([muAB, np.diag(sAB), np.diag(sAB), A, A.conj()],
              [[1, 2], [3, 1], [5, 2], [-1, 4, 3], [-2, 4, 5]])
    muBA = muBA / np.trace(muBA)

    return muAB, muBA


def orthog_MPS(sigBA, muBA, B, sBA, A, dtol=1e-12):
    """ set the MPS gauge across B-A link to the canonical form """
    # diagonalize left environment matrix
    dtemp, utemp = LA.eigh(sigBA)
    chitemp = sum(dtemp > dtol)
    DL = dtemp[range(-1, -chitemp - 1, -1)]
    UL = utemp[:, range(-1, -chitemp - 1, -1)]

    # diagonalize right environment matrix
    dtemp, utemp = LA.eigh(muBA)
    chitemp = sum(dtemp > dtol)
    DR = dtemp[range(-1, -chitemp - 1, -1)]
    UR = utemp[:, range(-1, -chitemp - 1, -1)]

    # compute new weights for B-A link
    weighted_mat = (np.diag(np.sqrt(DL)) @ UL.T @ np.diag(sBA)
                  @ UR @ np.diag(np.sqrt(DR)))
    UBA, stemp, VhBA = LA.svd(weighted_mat, full_matrices=False)
    sBA = stemp / LA.norm(stemp)

    # build x,y gauge change matrices, implement gauge change on A and B
    x = np.conj(UL) @ np.diag(1 / np.sqrt(DL)) @ UBA
    y = np.conj(UR) @ np.diag(1 / np.sqrt(DR)) @ VhBA.T
    A = ncon([y, A], [[1, -1], [1, -2, -3]])
    B = ncon([B, x], [[-1, -2, 2], [2, -3]])

    return B, sBA, A


def apply_gate_MPS(gateAB, A, sAB, B, sBA, chi, stol=1e-7):
    """ apply a gate to an MPS across and a A-B link. Truncate the MPS back to
    some desired dimension chi"""

    # ensure singular values are above tolerance threshold
    sBA_trim = sBA * (sBA > stol) + stol * (sBA < stol)

    # contract gate into the MPS, then deompose composite tensor with SVD
    d = A.shape[1]
    chiBA = sBA_trim.shape[0]
    tensors = [np.diag(sBA_trim), A, np.diag(sAB), B, np.diag(sBA_trim), gateAB]
    connects = [[-1, 1], [1, 5, 2], [2, 4], [4, 6, 3], [3, -4], [-2, -3, 5, 6]]
    nshape = [d * chiBA, d * chiBA]
    utemp, stemp, vhtemp = LA.svd(ncon(tensors, connects).reshape(nshape),
                                full_matrices=False)

    # truncate to reduced dimension
    chitemp = min(chi, len(stemp))
    utemp = utemp[:, range(chitemp)].reshape(sBA_trim.shape[0], d * chitemp)
    vhtemp = vhtemp[range(chitemp), :].reshape(chitemp * d, chiBA)

    # remove environment weights to form new MPS tensors A and B
    A = (np.diag(1 / sBA_trim) @ utemp).reshape(sBA_trim.shape[0], d, chitemp)
    B = (vhtemp @ np.diag(1 / sBA_trim)).reshape(chitemp, d, chiBA)

    # new weights
    sAB = stemp[range(chitemp)] / LA.norm(stemp[range(chitemp)])

    return A, sAB, B


def loc_density_MPS(A, sAB, B, sBA):
    """ Compute the local reduced density matrices from an MPS (assumend to be
    in canonical form)."""

    # recast singular weights into a matrix
    mAB = np.diag(sAB)
    mBA = np.diag(sBA)

    # contract MPS for local reduced density matrix (A-B)
    tensors = [np.diag(sBA**2), A, A.conj(), mAB, mAB, B, B.conj(),
             np.diag(sBA**2)]
    connects = [[3, 4], [3, -3, 1], [4, -1, 2], [1, 7], [2, 8], [7, -4, 5],
              [8, -2, 6], [5, 6]]
    rhoAB = ncon(tensors, connects)

    # contract MPS for local reduced density matrix (B-A)
    tensors = [np.diag(sAB**2), B, B.conj(), mBA, mBA, A, A.conj(),
             np.diag(sAB**2)]
    connects = [[3, 4], [3, -3, 1], [4, -1, 2], [1, 7], [2, 8], [7, -4, 5],
              [8, -2, 6], [5, 6]]
    rhoBA = ncon(tensors, connects)

    return rhoAB, rhoBA

In [17]:
import numpy as np
from ncon import ncon

""" Example 1: XX model """

# set bond dimensions and simulation options
chi = 16  # bond dimension
tau = 0.1  # timestep

numiter = 500  # number of timesteps
evotype = "imag"  # real or imaginary time evolution
E0 = -4 / np.pi  # specify exact ground energy (if known)
midsteps = int(1 / tau)  # timesteps between MPS re-orthogonalization

# define Hamiltonian (quantum XX model)
sX = np.array([[0, 1], [1, 0]])
sY = np.array([[0, -1j], [1j, 0]])
sZ = np.array([[1, 0], [0, -1]])
hamAB = (np.real(np.kron(sX, sX) + np.kron(sY, sY))).reshape(2, 2, 2, 2)
hamBA = (np.real(np.kron(sX, sX) + np.kron(sY, sY))).reshape(2, 2, 2, 2)

# initialize tensors
d = hamAB.shape[0]
sAB = np.ones(chi) / np.sqrt(chi)
sBA = np.ones(chi) / np.sqrt(chi)
A = np.random.rand(chi, d, chi)
B = np.random.rand(chi, d, chi)

""" Imaginary time evolution with TEBD """
# run TEBD routine
A, B, sAB, sBA, rhoAB, rhoBA = doTEBD(hamAB, hamBA, A, B, sAB, sBA, chi,
    tau, evotype=evotype, numiter=numiter, midsteps=midsteps, E0=E0)

# continute running TEBD routine with reduced timestep
tau = 0.01
numiter = 2000
midsteps = 100
A, B, sAB, sBA, rhoAB, rhoBA = doTEBD(hamAB, hamBA, A, B, sAB, sBA, chi,
    tau, evotype=evotype, numiter=numiter, midsteps=midsteps, E0=E0)

# continute running TEBD routine with reduced timestep and increased bond dim
chi = 32
tau = 0.001
numiter = 20000
midsteps = 1000
A, B, sAB, sBA, rhoAB, rhoBA = doTEBD(hamAB, hamBA, A, B, sAB, sBA, chi,
    tau, evotype=evotype, numiter=numiter, midsteps=midsteps, E0=E0)

# compare with exact results
energyMPS = np.real(0.5 * ncon([hamAB, rhoAB], [[1, 2, 3, 4], [1, 2, 3, 4]]) +
                    0.5 * ncon([hamBA, rhoBA], [[1, 2, 3, 4], [1, 2, 3, 4]]))
enErr = abs(energyMPS - E0)
print('Final results => Bond dim: %d, Energy: %f, Energy Error: %e' %
      (chi, energyMPS, enErr))

iteration: 0 of 500, chi: 16, t-step: 0.100000, energy: 3.834395, energy error: 5.107634e+00
iteration: 10 of 500, chi: 16, t-step: 0.100000, energy: -1.229575, energy error: 4.366435e-02
iteration: 20 of 500, chi: 16, t-step: 0.100000, energy: -1.237678, energy error: 3.556178e-02
iteration: 30 of 500, chi: 16, t-step: 0.100000, energy: -1.238375, energy error: 3.486459e-02
iteration: 40 of 500, chi: 16, t-step: 0.100000, energy: -1.238517, energy error: 3.472265e-02
iteration: 50 of 500, chi: 16, t-step: 0.100000, energy: -1.238532, energy error: 3.470788e-02
iteration: 60 of 500, chi: 16, t-step: 0.100000, energy: -1.238513, energy error: 3.472640e-02
iteration: 70 of 500, chi: 16, t-step: 0.100000, energy: -1.238488, energy error: 3.475181e-02
iteration: 80 of 500, chi: 16, t-step: 0.100000, energy: -1.238464, energy error: 3.477602e-02
iteration: 90 of 500, chi: 16, t-step: 0.100000, energy: -1.238443, energy error: 3.479670e-02
iteration: 100 of 500, chi: 16, t-step: 0.100000, en

iteration: 13000 of 20000, chi: 32, t-step: 0.001000, energy: -1.272793, energy error: 4.466304e-04
iteration: 14000 of 20000, chi: 32, t-step: 0.001000, energy: -1.272794, energy error: 4.459306e-04
iteration: 15000 of 20000, chi: 32, t-step: 0.001000, energy: -1.272794, energy error: 4.453992e-04
iteration: 16000 of 20000, chi: 32, t-step: 0.001000, energy: -1.272795, energy error: 4.449944e-04
iteration: 17000 of 20000, chi: 32, t-step: 0.001000, energy: -1.272795, energy error: 4.446855e-04
iteration: 18000 of 20000, chi: 32, t-step: 0.001000, energy: -1.272795, energy error: 4.444494e-04
iteration: 19000 of 20000, chi: 32, t-step: 0.001000, energy: -1.272795, energy error: 4.442685e-04
iteration: 20000 of 20000, chi: 32, t-step: 0.001000, energy: -1.272795, energy error: 4.441300e-04
Final results => Bond dim: 32, Energy: -1.272795, Energy Error: 4.441300e-04
